In [1]:
cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.6 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.6 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [2]:
cat /etc/nv_tegra_release

# R32 (release), REVISION: 7.3, GCID: 31982016, BOARD: t210ref, EABI: aarch64, DATE: Tue Nov 22 17:30:08 UTC 2022


In [3]:
uname -a

Linux jetson-nano 4.9.299-tegra #1 SMP PREEMPT Sun Jun 18 04:26:14 UTC 2023 aarch64 aarch64 aarch64 GNU/Linux


In [11]:
export TARGET=/workspaces/jetson_nano_kvm
export JETSON_NANO_KERNEL_SOURCE=$TARGET/Linux_for_Tegra/source/public
#export JETSON_NANO_KERNEL_SOURCE=$TARGET/Linux_for_Tegra/source/public/kernel_src

# 1.Set the shell variable with the command:
export TEGRA_KERNEL_OUT=${JETSON_NANO_KERNEL_SOURCE:?}/build
# Where:
#   <outdir> is the desired destination for the compiled kernel.
export KERNEL_MODULES_OUT=${JETSON_NANO_KERNEL_SOURCE:?}/modules

# 2.If cross-compiling on a non-Jetson system, export the following environment variables:
#export CROSS_COMPILE="$TARGET"/l4t-gcc/gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu/bin/aarch64-linux-gnu-
#export LOCALVERSION=-tegra
# Where:
#   <cross_prefix> is the absolute path of the ARM64 toolchain without the gcc suffix.
#   For example, for the reference ARM64 toolchain, <cross_prefix> is:
#     <toolchain_install_path>/bin/aarch64-linux-gnu-
# See The L4T Toolchain for information on how to download and build the reference toolchains.
# Note: NVIDIA recommends using the Linaro 7.3.1 2018.05 toolchain.

In [25]:
cd $TARGET || exit
ls -A "$TARGET"
xargs -a passwd.txt | sudo -S pwd

dotnet-install.sh  .gitignore       public_sources.tbz2  Readme.sh
dotnet.sh          Linux_for_Tegra  Readme.dib           .vscode
.git               passwd.txt       Readme.ipynb
[sudo] password for jetson: /workspaces/jetson_nano_kvm


In [6]:
rm -rf "${JETSON_NANO_KERNEL_SOURCE:?}"
rm -rf "$TARGET/l4t-gcc"
rm "$TARGET"/public_sources.tbz2
rm "$TARGET"/kernel*.tar.gz

rm: cannot remove '/workspaces/jetson_nano_kvm/public_sources.tbz2': No such file or directory


In [20]:
env | grep -E 'TARGET|JETSON_NANO_KERNEL_SOURCE|TEGRA_KERNEL_OUT|KERNEL_MODULES_OUT|CROSS_COMPILE|LOCALVERSION' | sort

JETSON_NANO_KERNEL_SOURCE=/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public
KERNEL_MODULES_OUT=/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/modules
TARGET=/workspaces/jetson_nano_kvm
TEGRA_KERNEL_OUT=/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build


In [13]:
sudo apt update && sudo apt-get install -y build-essential bc git curl wget xxd kmod libssl-dev

Get:1 file:/var/cuda-repo-l4t-10-2-local  InRelease
Ign:1 file:/var/cuda-repo-l4t-10-2-local  InRelease
Get:2 file:/var/visionworks-repo  InRelease
Ign:2 file:/var/visionworks-repo  InRelease
Get:3 file:/var/visionworks-sfm-repo  InRelease
Ign:3 file:/var/visionworks-sfm-repo  InRelease
Get:4 file:/var/visionworks-tracking-repo  InRelease
Ign:4 file:/var/visionworks-tracking-repo  InRelease
Get:5 file:/var/cuda-repo-l4t-10-2-local  Release [564 B]
Get:6 file:/var/visionworks-repo  Release [2,001 B]
Get:7 file:/var/visionworks-sfm-repo  Release [2,005 B]                        
Get:8 file:/var/visionworks-tracking-repo  Release [2,010 B]                   
Get:5 file:/var/cuda-repo-l4t-10-2-local  Release [564 B]                      
Get:6 file:/var/visionworks-repo  Release [2,001 B]                            
Get:7 file:/var/visionworks-sfm-repo  Release [2,005 B]                        
Get:8 file:/var/visionworks-tracking-repo  Release [2,010 B]                   
Get:9 https://re

In [14]:
# Gets the kernel
#wget https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.1/Sources/T210/public_sources.tbz2  # 32.7.1
wget -O "$TARGET"/public_sources.tbz2 https://developer.nvidia.com/downloads/remack-sdksjetpack-463r32releasev73sourcest210publicsourcestbz2 # 32.7.3
cd "$TARGET" || exit
tar -xf "$TARGET"/public_sources.tbz2

--2023-06-18 23:37:12--  https://developer.nvidia.com/downloads/remack-sdksjetpack-463r32releasev73sourcest210publicsourcestbz2
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.39.144
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/assets/embedded/secure/tools/files/jetpack-sdks/jetpack-4.6.3/r32_Release_v7.3/sources/T210/public_sources.tbz2?bZkhPaspIuQJJd7BbD598k2Fe7TCkT3iV4IOsiMmNVxJW3jZC0RJdPKq_OzDuyCQAEr0qwfi8vv98mQN1RBiZplkv6-rDip5rkmzoBUn08U3OJUbLiONeAHhsthdY1ZsMr5CpvNAD8oVeLp0kh-ypai2xTE-c9HO2ctN2fr8UjkApwRSlzRQ5gGTLNkaO0nKXLqfxs8VZFacdV2yK2DrB4ZlX2U-8b-SRFMALGZO [following]
--2023-06-18 23:37:13--  https://developer.download.nvidia.com/assets/embedded/secure/tools/files/jetpack-sdks/jetpack-4.6.3/r32_Release_v7.3/sources/T210/public_sources.tbz2?bZkhPaspIuQJJd7BbD598k2Fe7TCkT3iV4IOsiMmNVxJW3jZC0RJdPKq_OzDuyCQAEr0qwfi8vv98m

In [15]:
ls -A $JETSON_NANO_KERNEL_SOURCE
cd $JETSON_NANO_KERNEL_SOURCE || exit
ls -la $JETSON_NANO_KERNEL_SOURCE/kernel_src.tbz2
tar -xf $JETSON_NANO_KERNEL_SOURCE/kernel_src.tbz2

dtc-1.4.0.tar.bz2                    gstomx1_src.tbz2.sha1sum
dtc-1.4.0.tar.bz2.sha1sum            kernel_src.tbz2
FreeRTOSV8.1.2_src.tbz2              kernel_src.tbz2.sha1sum
FreeRTOSV8.1.2_src.tbz2.sha1sum      libgstnvdrmvideosink_src.tbz2
gstegl_src.tbz2                      libgstnvdrmvideosink_src.tbz2.sha1sum
gstegl_src.tbz2.sha1sum              libgstnvvideosinks_src.tbz2
gstjpeg_src.tbz2                     libgstnvvideosinks_src.tbz2.sha1sum
gstjpeg_src.tbz2.sha1sum             nvgstapps_src.tbz2
gst-nvarguscamera_src.tbz2           nvgstapps_src.tbz2.sha1sum
gst-nvarguscamera_src.tbz2.sha1sum   nvidia-xconfig_src.tbz2
gst-nvcompositor_src.tbz2            nvidia-xconfig_src.tbz2.sha1sum
gst-nvcompositor_src.tbz2.sha1sum    nv_public_src_build.sh
gst-nvtee_src.tbz2                   nvsample_cudaprocess_src.tbz2
gst-nvtee_src.tbz2.sha1sum           nvsample_cudaprocess_src.tbz2.sha1sum
gst-nvv4l2camera_src.tbz2            opencv_gst_samples_src.tbz2
gst-nvv4l2camera_src.tbz2.s

In [17]:
ls -AF $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9
cd $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9 || exit

arch/                            crypto/                 mm/
block/                           Documentation/          net/
build.config.aarch64             drivers/                NVIDIA-REVIEWERS
build.config.common              firmware/               README
build.config.cuttlefish.aarch64  fs/                     REPORTING-BUGS
build.config.cuttlefish.x86_64   .get_maintainer.ignore  rt-patches/
build.config.goldfish.arm        .gitattributes          samples/
build.config.goldfish.arm64      include/                scripts/
build.config.goldfish.mips       init/                   security/
build.config.goldfish.mips64     ipc/                    signing_key
build.config.goldfish.x86        Kbuild                  signing_key.x509
build.config.goldfish.x86_64     Kconfig                 sound/
build.config.x86_64              kernel/                 tools/
certs/                           lib/                    usr/
.cocciconfig                     .mailmap                verity_de

In [18]:
grep -A5 '0x0 0x50041000 0x0 0x1000' -r $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc
sed -i.orig -e 's/0x0 0x50042000 0x0 0x0100>;/0x0 0x50042000 0x0 0x2000\n\t\t       0x0 0x50044000 0x0 0x2000\n\t\t       0x0 0x50046000 0x0 0x2000>;\n\t\tinterrupts = <GIC_PPI 9 (GIC_CPU_MASK_SIMPLE(4) | IRQ_TYPE_LEVEL_HIGH)>;/' \
  $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi
diff -u $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi{.orig,}

/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi:		reg = <0x0 0x50041000 0x0 0x1000
/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-		       0x0 0x50042000 0x0 0x0100>;
/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-		status = "disabled";
/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-	};
/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-
/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-	lic: interrupt-controller@60004000 {
--- /workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/

: 1

In [22]:
cd $JETSON_NANO_KERNEL_SOURCE || exit
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" LOCALVERSION=-tegra tegra_defconfig
cp "${TEGRA_KERNEL_OUT:?}"/.config{,.orig}

make: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  HOSTCC  scripts/basic/fixdep
  GEN     ./Makefile
  HOSTCC  scripts/kconfig/conf.o
  SHIPPED scripts/kconfig/zconf.tab.c
  SHIPPED scripts/kconfig/zconf.lex.c
  SHIPPED scripts/kconfig/zconf.hash.c
  HOSTCC  scripts/kconfig/zconf.tab.o
  HOSTLD  scripts/kconfig/conf
#
# configuration written to .config
#
make[1]: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'


In [23]:
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" menuconfig

make: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  GEN     ./Makefile
  HOSTCC  scripts/kconfig/mconf.o
  HOSTCC  scripts/kconfig/lxdialog/checklist.o
  HOSTCC  scripts/kconfig/lxdialog/util.o
  HOSTCC  scripts/kconfig/lxdialog/inputbox.o
  HOSTCC  scripts/kconfig/lxdialog/textbox.o
  HOSTCC  scripts/kconfig/lxdialog/yesno.o
  HOSTCC  scripts/kconfig/lxdialog/menubox.o
  HOSTLD  scripts/kconfig/mconf
scripts/kconfig/mconf  Kconfig
Error opening terminal: unknown.
/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9/scripts/kconfig/Makefile:28: recipe for target 'menuconfig' failed
make[2]: *** [menuconfig] Error 1
/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9/Makefile:575: recipe for target 'menuconfig' failed
make[1]: *** [menuconfig] Error 2
make[1]: Leaving directory '/workspaces/jetso

: 2

In [27]:
cd $JETSON_NANO_KERNEL_SOURCE || exit

In [28]:
# Generates the Image that we're gonna place on /boot/Image
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" LOCALVERSION=-tegra -j"$(nproc)" --output-sync=target zImage

  GEN     ./Makefile
scripts/kconfig/conf  --silentoldconfig Kconfig
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  GEN     ./Makefile
make[1]: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  CHK     include/config/kernel.release
  UPD     include/config/kernel.release
make[1]: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  WRAP    arch/arm64/include/generated/asm/clkdev.h
  WRAP    arch/arm64/include/generated/asm/delay.h
  WRAP    arch/arm64/include/generated/asm/bugs.h
  WRAP    arch/arm64/include/generated/asm/dma.h
  WRAP    arch/arm64/include/generated/asm/cputime.h
  WRAP    arch/arm64/include/generated/asm/div64.h
  WRAP    arch/arm64/include/generated/asm/dma-contiguous.h
  WRAP    arch/arm64/include/generated/asm/early_ioremap.h
  WRAP    arch/arm64/include/ge

In [29]:
# Generates the drivers. This is needed because the old driver will not work with our new Image
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" LOCALVERSION=-tegra -j"$(nproc)" --output-sync=target modules

make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  GEN     ./Makefile
make[1]: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  CHK     include/config/kernel.release
make[1]: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  CHK     include/generated/uapi/linux/version.h
make[1]: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  CHK     include/generated/utsrelease.h
make[1]: Leaving directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make[1]: Entering directory '/workspaces/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  Using

In [ ]:
# Generates our modified device file trees
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" LOCALVERSION=-tegra -j"$(nproc)" --output-sync=target dtbs

In [ ]:
# Installs the modules on the build folder $TARGET/Linux_for_Tegra/source/public/build
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" LOCALVERSION=-tegra INSTALL_MOD_PATH="${KERNEL_MODULES_OUT:?}" modules_install